In [ ]:
# EDA

In [ ]:
## Text Lenghts

In [ ]:
# Visualize the distribution of text lengths

In [ ]:
plt.figure(figsize=(10, 6))

In [ ]:
sns.histplot(df['cleaned_length'], bins=30, color='purple')

In [ ]:
plt.title('Distribution of Text Lengths')

In [ ]:
plt.xlabel('Length of Text (number of words)')

In [ ]:
plt.ylabel('Frequency')

In [ ]:
plt.show()

In [ ]:
import pandas as pd

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Assuming 'df' is your DataFrame and 'cleaned_text' is the column with the text data

In [ ]:
vectorizer = CountVectorizer()

In [ ]:
X = vectorizer.fit_transform(df['cleaned_text'])

In [ ]:
# Summing up the counts of each vocabulary word

In [ ]:
sum_words = X.sum(axis=0)

In [ ]:
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]

In [ ]:
words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)

In [ ]:
# Extracting the top 100 words

In [ ]:
top_words = words_freq[:60]

In [ ]:
# Converting to DataFrame for better visualization

In [ ]:
top_words_df = pd.DataFrame(top_words, columns=['Word', 'Frequency'])

In [ ]:
# Printing the top 100 words with their frequencies

In [ ]:
top_words_df

In [ ]:
## Calculate and Visualize the Frequency of Individual Words

In [ ]:
from collections import Counter

In [ ]:
import itertools

In [ ]:
# Count frequencies of all words in the cleaned text

In [ ]:
all_words = list(itertools.chain(*df['cleaned_text'].str.split()))

In [ ]:
word_counts = Counter(all_words)

In [ ]:
# Display the top 20 most common words

In [ ]:
most_common_words = word_counts.most_common(20)

In [ ]:
words, counts = zip(*most_common_words)

In [ ]:
# Corrected code to visualize the Top 20 Most Frequent Words

In [ ]:
plt.figure(figsize=(12, 8))

In [ ]:
sns.barplot(x=list(words), y=list(counts))  # Explicitly name the parameters

In [ ]:
plt.title('Top 20 Most Frequent Words')

In [ ]:
plt.xticks(rotation=45)

In [ ]:
plt.xlabel('Words')

In [ ]:
plt.ylabel('Frequency')

In [ ]:
plt.show()

In [ ]:
## Visualize the Sentiment Distribution

In [ ]:
print(df.columns)

In [ ]:
sns.countplot(x=df['class'])

In [ ]:
plt.title('Distribution of Sentiment Labels')

In [ ]:
plt.xlabel('Sentiment')

In [ ]:
plt.ylabel('Count')

In [ ]:
plt.show()

In [ ]:
# Calculate class imbalance ratio

In [ ]:
positive_count = df[df['class'] == 'positive'].shape[0]

In [ ]:
negative_count = df[df['class'] == 'negative'].shape[0]

In [ ]:
ratio = positive_count / negative_count

In [ ]:
print(f"Ratio of Positive to Negative Samples: {ratio:.2f}")

In [ ]:
sentiment_counts = df['class'].value_counts()

In [ ]:
plt.figure(figsize=(8, 8))

In [ ]:
plt.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=140, colors=['skyblue', 'orange', 'green'])

In [ ]:
plt.title('Pie Chart of Sentiment Distribution')

In [ ]:
plt.show()

In [ ]:
## Generate Word Clouds for Each Sentiment Category

In [ ]:
from wordcloud import WordCloud

In [ ]:
def generate_wordcloud(data, title):

In [ ]:
    text = ' '.join(data)

In [ ]:
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

In [ ]:
    plt.figure(figsize=(10, 5))

In [ ]:
    plt.imshow(wordcloud, interpolation='bilinear')

In [ ]:
    plt.axis('off')

In [ ]:
    plt.title(title)

In [ ]:
    plt.show()

In [ ]:
for sentiment in ['positive', 'neutral', 'negative']:

In [ ]:
    generate_wordcloud(df[df['class'] == sentiment]['cleaned_text'], f'Word Cloud for {sentiment.capitalize()} Sentiment')

In [ ]:
## Analyze the Frequency of Bigrams

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(ngram_range=(2, 2))

In [ ]:
bigrams = vectorizer.fit_transform(df['cleaned_text'])

In [ ]:
bigram_counts = bigrams.sum(axis=0)

In [ ]:
bigram_frequencies = [(word, bigram_counts[0, idx]) for word, idx in vectorizer.vocabulary_.items()]

In [ ]:
bigram_frequencies = sorted(bigram_frequencies, key=lambda x: x[1], reverse=True)[:20]

In [ ]:
# Plotting the Top 20 Bigrams

In [ ]:
words, counts = zip(*bigram_frequencies)

In [ ]:
plt.figure(figsize=(12, 8))

In [ ]:
sns.barplot(x=list(words), y=list(counts))

In [ ]:
plt.title('Top 20 Most Frequent Bigrams')

In [ ]:
plt.xticks(rotation=90)

In [ ]:
plt.xlabel('Bigrams')

In [ ]:
plt.ylabel('Frequency')

In [ ]:
plt.show()

In [ ]:
## Analyze the Frequency of Trigrams

In [ ]:
vectorizer = CountVectorizer(ngram_range=(3, 3))

In [ ]:
trigrams = vectorizer.fit_transform(df['cleaned_text'])

In [ ]:
trigram_counts = trigrams.sum(axis=0)

In [ ]:
trigram_frequencies = [(word, trigram_counts[0, idx]) for word, idx in vectorizer.vocabulary_.items()]

In [ ]:
trigram_frequencies = sorted(trigram_frequencies, key=lambda x: x[1], reverse=True)[:50]

In [ ]:
# Plotting the top 20 trigrams

In [ ]:
words, counts = zip(*trigram_frequencies)

In [ ]:
plt.figure(figsize=(15, 10))

In [ ]:
sns.barplot(x=list(words), y=list(counts))

In [ ]:
plt.title('Top 20 Most Frequent Trigrams')

In [ ]:
plt.xticks(rotation=90)

In [ ]:
plt.xlabel('Trigrams')

In [ ]:
plt.ylabel('Frequency')

In [ ]:
plt.show()

In [ ]:
## Co-Occurrence Networks for Words in the Text

In [ ]:
import networkx as nx

In [ ]:
from itertools import combinations

In [ ]:
def create_cooccurrence_matrix(line, vocab):

In [ ]:
    co_occurrences = nx.Graph()

In [ ]:
    for word1, word2 in combinations(line.split(), 2):

In [ ]:
        if word1 in vocab and word2 in vocab:

In [ ]:
            if co_occurrences.has_edge(word1, word2):

In [ ]:
                co_occurrences[word1][word2]['weight'] += 1

In [ ]:
            else:

In [ ]:
                co_occurrences.add_edge(word1, word2, weight=1)

In [ ]:
    return co_occurrences

In [ ]:
# Create a graph for the top 20 most frequent words

In [ ]:
top_words = set([word for word, count in word_counts.most_common(20)])

In [ ]:
co_occurrence_graph = nx.Graph()